In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              # "6_Hr_BESS_Fuelx2",
            #   "8_Hr_BESS",
            #   "8_Hr_BESS_Fuelx2",
            #   "10_Hr_BESS",
            #   "10_Hr_BESS_Fuelx2",
              ]

In [3]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [4]:
plots_path = os.path.join(current_dir, 'plots') + "/"
tables_path = os.path.join(current_dir, 'tables') + "/"

In [5]:
# modeling scaling ModelScalingFactor
ModelScalingFactor = 1000

cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

params_names = ['Inv_cost_MW', 'Inv_cost_MWh', 'Fixed_OM_cost_MW', 'Var_OM_cost_out', 
          'Fuel_cost', 'Var_OM_cost_in', 'StartCost', 'Charge_cost', 'EnergyRevenue', 
          'OperatingReserveRevenue', 'OperatingRegulationRevenue',]

param_print_names = params_names

for i in range(len(param_print_names)):
    param_print_names[i] = param_print_names[i].replace("_", " ")

cost_params = ['Inv_cost_MW', 'Inv_cost_MWh', 'Fixed_OM_cost_MW', 'Var_OM_cost_out', 
        'Fuel_cost', 'Var_OM_cost_in', 'StartCost', 'Charge_cost']

revenue_params = ['EnergyRevenue', 'OperatingReserveRevenue', 'OperatingRegulationRevenue']

In [6]:
# get load data

load_data_raw = pd.read_csv(cem_path + '\\Thermal_Base\\' + 'system\\' + 'Demand_data.csv')
load_data = load_data_raw['Demand_MW_z1']

In [7]:
date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

filename = 'operational_profits_' + date + '.pdf'

In [8]:
unit_profit_dic = {case: None for case in case_names}
percentage_profits_dic = {case: None for case in case_names}
consumer_payments_dic = {case: None for case in case_names}
total_investment_costs_dic = {case: None for case in case_names}
total_operating_costs_dic = {case: None for case in case_names}
total_system_costs_dic = {case: None for case in case_names} 
total_inv_op_costs_dic = {case: None for case in case_names}
total_objective_dic = {case: None for case in case_names}
nse_costs_dic = {case: None for case in case_names}
total_nse_dic = {case: None for case in case_names}
total_unmet_rsv_dic = {case: None for case in case_names}
total_nse_cost_dic = {case: None for case in case_names}
total_unmet_rsv_cost_dic = {case: None for case in case_names}
total_fixed_om_costs_dic = {case: None for case in case_names}
total_var_om_out_costs_dic = {case: None for case in case_names}
total_fuel_costs_dic = {case: None for case in case_names}
total_var_om_in_costs_dic = {case: None for case in case_names}
total_start_costs_dic = {case: None for case in case_names}
total_charge_costs_dic = {case: None for case in case_names}

In [9]:
vre_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Vre.csv')

In [10]:
vre_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Vre.csv')
thermal_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Thermal.csv')
storage_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Storage.csv')

In [11]:
dfGen = pd.concat([vre_dfGen, thermal_dfGen, storage_dfGen], ignore_index=True)

In [12]:
dfGen

,Resource,Zone,Num_VRE_Bins,New_Build,Can_Retire,Existing_Cap_MW,Max_Cap_MW,Min_Cap_MW,Inv_Cost_per_MWyr,Fixed_OM_Cost_per_MWyr,...,Max_Cap_MWh,Min_Cap_MWh,Inv_Cost_per_MWhyr,Fixed_OM_Cost_per_MWhyr,Var_OM_Cost_per_MWh_In,Self_Disch,Eff_Up,Eff_Down,Min_Duration,Max_Duration
0,Land-Based Wind - Class 1 - Technology 1,1,1.0,1,0,32791.061172,-1,0,83289.329169,32443.047267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Utility PV - Class 1,1,1.0,1,0,17972.934821,-1,0,74134.164015,23765.603456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NG 2-on-1 Combined Cycle (F-Frame),1,NaN,1,0,46274.023418,-1,0,56221.989154,25875.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NG Combustion Turbine (F-Frame),1,NaN,1,0,9457.159921,-1,0,50580.567617,19575.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
for case_name in case_names:

    print('Case Name: ' + case_name + '\n')

    # load generator characteristics from resources folder
    thermal_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Thermal.csv')
    vre_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Vre.csv')
    storage_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Storage.csv')

    # combine all resources to dfGen
    dfGen = pd.concat([thermal_dfGen, vre_dfGen, storage_dfGen,], ignore_index=True)

    gen_capacity_gw = dfGen['Existing_Cap_MW'] / ModelScalingFactor
    gen_capacity_mw = dfGen['Existing_Cap_MW']
    gen_capacity_mwhour = dfGen['Existing_Cap_MWh']
    gen_capacity_mwhour = dfGen['Existing_Cap_MWh'] / ModelScalingFactor

    resource_list = dfGen['Resource']

    # solar and wind id
    solar_id = dfGen[dfGen['Resource'] == 'Utility PV - Class 1'].index[0]
    wind_id = dfGen[dfGen['Resource'] == 'Land-Based Wind - Class 1 - Technology 1'].index[0]

    cem_case = 'CEM_' + case_name + '_ABB\\'
    pf_case = 'PF_' + case_name + '\\'
    dlac_case = 'DLAC_' + case_name + '\\'
    dlac_imperfect_case = 'DLAC_imperfect_' + case_name + '\\'
    slac_case = 'SLAC_' + case_name + '\\'

    cem_results_path = cem_path + '\\' + case_name + '\\' + 'results\\'
    pf_results_path = policies_path + '\\' + case_name + '\\' + 'results_pf\\'
    dlac_results_path = policies_path + '\\' + case_name + '\\' + 'results_dlac-p\\'
    dlac_imperfect_results_path = policies_path + '\\' + case_name + '\\' + 'results_dlac-i\\'
    slac_results_path = policies_path + '\\' + case_name + '\\' + 'results_slac\\'

    cem_raw = pd.read_csv(cem_results_path + 'NetRevenue.csv')
    pf_raw = pd.read_csv(pf_results_path + 'NetRevenue.csv')
    dlac_raw = pd.read_csv(dlac_results_path + 'NetRevenue.csv')
    dlac_imperfect_raw = pd.read_csv(dlac_imperfect_results_path + 'NetRevenue.csv')
    slac_raw = pd.read_csv(slac_results_path + 'NetRevenue.csv')

    ### XXX Fix inv cost MWh
    # dlac_raw['Inv_cost_MWh'] = cem_raw['Inv_cost_MWh']
    # dlac_imperfect_raw['Inv_cost_MWh'] = cem_raw['Inv_cost_MWh']
    # slac_raw['Inv_cost_MWh'] = cem_raw['Inv_cost_MWh']


    cem_raw_prices = pd.read_csv(cem_results_path + 'prices.csv')
    pf_raw_prices = pd.read_csv(pf_results_path + 'price_electricity.csv', header=None)
    dlac_raw_prices = pd.read_csv(dlac_results_path + 'price_electricity.csv', header=None)
    dlac_imperfect_raw_prices = pd.read_csv(dlac_imperfect_results_path + 'price_electricity.csv', header=None)
    slac_raw_prices = pd.read_csv(slac_results_path + 'price_electricity.csv', header=None)

    cem_prices = cem_raw_prices['1']
    pf_prices = pf_raw_prices[0]
    dlac_prices = dlac_raw_prices[0]
    dlac_imperfect_prices = dlac_imperfect_raw_prices[0]
    slac_prices = slac_raw_prices[0]

    column_names = cem_raw.columns[1:]
    ### get cem dataframe of only non-trivial resources
    cem_raw_upd = cem_raw[cem_raw['Resource'].isin(resource_list)]
    # make sure the order of the resources is the same in dfGen and cem_raw_upd
    cem_raw_upd = cem_raw_upd.set_index('Resource').reindex(resource_list).reset_index()


    gen_print_names = resource_list
    # print(gen_print_names)

    param_print_names = params_names

    for i in range(len(param_print_names)):
        param_print_names[i] = param_print_names[i].replace("_", " ")



    cem_raw_prices = pd.read_csv(cem_results_path + 'prices.csv')
    pf_raw_prices = pd.read_csv(pf_results_path + 'price_electricity.csv', header=None)
    dlac_raw_prices = pd.read_csv(dlac_results_path + 'price_electricity.csv', header=None)
    dlac_imperfect_raw_prices = pd.read_csv(dlac_imperfect_results_path + 'price_electricity.csv', header=None)
    slac_raw_prices = pd.read_csv(slac_results_path + 'price_electricity.csv', header=None)

    cem_economics = cem_raw_upd[cost_params + revenue_params]
    pf_economics = pf_raw[cost_params + revenue_params]
    dlac_economics = dlac_raw[cost_params + revenue_params]
    dlac_imperfect_economics = dlac_imperfect_raw[cost_params + revenue_params]
    slac_economics = slac_raw[cost_params + revenue_params]

    cem_economics_copy = cem_economics.copy()
    pf_economics_copy = pf_economics.copy()
    dlac_economics_copy = dlac_economics.copy()
    dlac_imperfect_economics_copy = dlac_imperfect_economics.copy()
    slac_economics_copy = slac_economics.copy()

    cem_economics_copy[cost_params] = cem_economics_copy[cost_params] * -1
    pf_economics_copy[cost_params] = pf_economics_copy[cost_params] * -1
    dlac_economics_copy[cost_params] = dlac_economics_copy[cost_params] * -1
    dlac_imperfect_economics_copy[cost_params] = dlac_imperfect_economics_copy[cost_params] * -1
    slac_economics_copy[cost_params] = slac_economics_copy[cost_params] * -1

    cem_profit = cem_economics_copy.sum(axis=1)
    pf_profit = pf_economics_copy.sum(axis=1)
    dlac_profit = dlac_economics_copy.sum(axis=1)
    dlac_imperfect_profit = dlac_imperfect_economics_copy.sum(axis=1)
    slac_profit = slac_economics_copy.sum(axis=1)

    cem_economics_scaled = cem_economics_copy / ModelScalingFactor**2
    pf_economics_scaled = pf_economics_copy / ModelScalingFactor**2
    dlac_economics_scaled = dlac_economics_copy / ModelScalingFactor**2
    dlac_imperfect_economics_scaled = dlac_imperfect_economics_copy / ModelScalingFactor**2
    slac_economics_scaled = slac_economics_copy / ModelScalingFactor**2

    cem_scaled_profit = cem_economics_scaled.sum(axis=1)
    pf_scaled_profit = pf_economics_scaled.sum(axis=1)
    dlac_scaled_profit = dlac_economics_scaled.sum(axis=1)
    dlac_imperfect_scaled_profit = dlac_imperfect_economics_scaled.sum(axis=1)
    slac_scaled_profit = slac_economics_scaled.sum(axis=1)

    ###================================================================================================
    ### Unit Profits
    ###================================================================================================
    cem_unit_profit_mw = cem_profit / gen_capacity_mw
    pf_unit_profit_mw = pf_profit / gen_capacity_mw
    dlac_unit_profit_mw = dlac_profit / gen_capacity_mw
    dlac_imperfect_unit_profit_mw = dlac_imperfect_profit / gen_capacity_mw
    slac_unit_profit_mw = slac_profit / gen_capacity_mw

    cem_unit_profit_mw_sci = cem_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    pf_unit_profit_mw_sci = pf_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    dlac_unit_profit_mw_sci = dlac_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    dlac_imperfect_unit_profit_mw_sci = dlac_imperfect_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    slac_unit_profit_mw_sci = slac_unit_profit_mw.apply(lambda x: f"{x:.4e}")

    cem_unit_profit_gw = cem_profit / gen_capacity_gw
    pf_unit_profit_gw = pf_profit / gen_capacity_gw
    dlac_unit_profit_gw = dlac_profit / gen_capacity_gw
    dlac_imperfect_unit_profit_gw = dlac_imperfect_profit / gen_capacity_gw
    slac_unit_profit_gw = slac_profit / gen_capacity_gw

    unit_profit_df = pd.DataFrame({
        'Resource': gen_print_names,
        'CEM': cem_unit_profit_mw_sci,
        'PF': pf_unit_profit_mw_sci,
        'DLAC': dlac_unit_profit_mw_sci,
        'DLAC Imperfect': dlac_imperfect_unit_profit_mw_sci,
        'SLAC': slac_unit_profit_mw_sci
    })

    # sorted_unit_profits = sorted(unit_profits.items(), key=lambda item: item[1], reverse=True)

    unit_profit_dic[case_name] = unit_profit_df

    ###================================================================================================
    ### Calculate consumer payments
    ###================================================================================================

    cem_consumer_payments = cem_prices * load_data # - nse?
    pf_consumer_payments = pf_prices * load_data # - nse?
    dlac_consumer_payments = dlac_prices * load_data # - nse?
    dlac_imperfect_consumer_payments = dlac_imperfect_prices * load_data  # - nse?
    slac_consumer_payments = slac_prices * load_data # - nse?

    sum_cem_cp = cem_consumer_payments.sum() + cem_reg_revenue_sum + cem_rsv_revenue_sum
    sum_pf_cp = pf_consumer_payments.sum() + pf_reg_revenue_sum + pf_rsv_revenue_sum
    sum_dlac_cp = dlac_consumer_payments.sum() + dlac_reg_revenue_sum + dlac_rsv_revenue_sum
    sum_dlac_imperfect_cp = dlac_imperfect_consumer_payments.sum() + \
                                    dlac_imperfect_reg_revenue_sum + dlac_imperfect_rsv_revenue_sum
    sum_slac_cp = slac_consumer_payments.sum()  + slac_reg_revenue_sum + slac_rsv_revenue_sum

    # # Convert sums to millions of dollars
    # sum_cem_cp_mil = sum_cem_cp / 1e6
    # sum_dlac_cp_mil = sum_dlac_cp / 1e6
    # sum_dlac_imperfect_cp_mil = sum_dlac_imperfect_cp / 1e6
    # sum_slac_cp_mil = sum_slac_cp / 1e6

    sum_cem_cp_sci = f"{sum_cem_cp:.4e}"
    sum_pf_cp_sci = f"{sum_pf_cp:.4e}"
    sum_dlac_cp_sci = f"{sum_dlac_cp:.4e}"
    sum_dlac_imperfect_cp_sci = f"{sum_dlac_imperfect_cp:.4e}"
    sum_slac_cp_sci = f"{sum_slac_cp:.4e}"

    # Create a dictionary with the sums
    consumer_payments = {
        'CEM': sum_cem_cp_sci,
        'PF': sum_pf_cp_sci,
        'DLAC': sum_dlac_cp_sci,
        'DLAC Imperfect': sum_dlac_imperfect_cp_sci,
        'SLAC': sum_slac_cp_sci
    }

    # # Sort the dictionary by values in descending order
    # sorted_consumer_payments = sorted(consumer_payments.items(), key=lambda item: item[1], reverse=True)

    consumer_payments_dic[case_name] = consumer_payments

    ###================================================================================================
    ### Calculate Profit Margin Rate
    ###================================================================================================

    # dfGen.index = gen_print_names
    char_str = ['Inv_Cost_per_MWyr', 'Inv_Cost_per_MWhyr', 'Fixed_OM_Cost_per_MWyr']
    dfGen_annual_costs = dfGen[char_str]

    # if the Inv_Cost_per_MWhyr in each row is > 0, then multiply the Inv_Cost_per_MWhyr by the Max_Duration
    inv_cost_per_mwhyr2mwyr = dfGen.apply(
        lambda row: row['Inv_Cost_per_MWhyr'] * row['Existing_Cap_MWh'] / row['Existing_Cap_MW'] \
            if row['Inv_Cost_per_MWhyr'] > 0 else row['Inv_Cost_per_MWhyr'],
        axis=1
    )

    # Replace 'Inv_Cost_per_MWhyr' in dfGen_annual_costs with inv_cost_permwhyr2mwyr and title it 'Inv_Cost_per_Energy_by_hour'
    dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
    dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)

    dfGen_annual_costs_sum = dfGen_annual_costs.sum(axis=1) # $/MWyr

    # \alpha_{i,\Pi} = \frac{P_{i,\Pi}}{C_{i,\Pi}}
    cem_annual_profit2costs = cem_unit_profit_mw / dfGen_annual_costs_sum
    pf_annual_profit2costs = pf_unit_profit_mw / dfGen_annual_costs_sum
    dlac_annual_profit2costs = dlac_unit_profit_mw / dfGen_annual_costs_sum
    dlac_imperfect_annual_profit2costs = dlac_imperfect_unit_profit_mw / dfGen_annual_costs_sum
    slac_annual_profit2costs = slac_unit_profit_mw / dfGen_annual_costs_sum

    cem_ap2c_perc = cem_annual_profit2costs * 100
    pf_ap2c_perc = pf_annual_profit2costs * 100
    dlac_ap2c_perc = dlac_annual_profit2costs * 100
    dlac_imperfect_ap2c_perc = dlac_imperfect_annual_profit2costs * 100
    slac_ap2c_perc = slac_annual_profit2costs * 100

    cem_ap2c_perc_sci = cem_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    pf_ap2c_perc_sci = pf_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    dlac_ap2c_perc_sci = dlac_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    dlac_imperfect_perc_ap2c_sci = dlac_imperfect_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    slac_ap2c_perc_sci = slac_ap2c_perc.apply(lambda x: f"{x:.2f}%")


    percentage_profit_df = pd.DataFrame({
        'Resource': gen_print_names,
        'CEM': cem_ap2c_perc_sci,
        'PF': pf_ap2c_perc_sci,
        'DLAC': dlac_ap2c_perc_sci,
        'DLAC Imperfect': dlac_imperfect_perc_ap2c_sci,
        'SLAC': slac_ap2c_perc_sci
    })

    percentage_profits_dic[case_name] = percentage_profit_df
    ### DONE


Case Name: Thermal_Base



NameError: name 'cem_reg_revenue_sum' is not defined

In [ ]:
# Create a dictionary from the replace statements
simplify_resource_names = {
    "Coal-new": "Coal",
    "NG Combustion Turbine (F-Frame)": "NG CT",
    "NG 2-on-1 Combined Cycle (F-Frame)": "NG CC",
    "NG 2-on-1 Combined Cycle (H-Frame) 95% CCS": "NG CC",
    "Utility PV - Class 1": "Solar",
    "Land-Based Wind - Class 1 - Technology 1": "Wind",
    "PBR-HTGR": "SMR",
    "iPWR_Pack": "SMR",
    "HGTR": "SMR",
    "Microreactor": "SMR",
    "RICE": "IC",
    "Utility-Scale Battery Storage": "Battery"
}

# Function to replace values in a DataFrame based on the dictionary
def replace_strings_in_df_column(df, column_name, replace_dict):
    """
    Replace values in a specified column of a DataFrame based on a dictionary.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The column in which to replace values.
        replace_dict (dict): A dictionary where keys are old values and values are new values.

    Returns:
        pd.DataFrame: The DataFrame with replaced values.
    """
    df[column_name] = df[column_name].replace(replace_dict)
    return df

In [ ]:
def percent_change_df(df):
    norm = df.copy()
    norm['CEM'] = df['CEM'].astype(float) / df['PF'].astype(float)
    norm['PF'] = df['PF'].astype(float) / df['PF'].astype(float)
    norm['DLAC'] = df['DLAC'].astype(float) / df['PF'].astype(float)
    norm['DLAC Imperfect'] = df['DLAC Imperfect'].astype(float) / df['PF'].astype(float)
    norm['SLAC'] = df['SLAC'].astype(float) / df['PF'].astype(float)

    change_df = norm.copy()
    change_df['CEM'] = norm['CEM'] - 1
    change_df['PF'] = norm['PF'] - 1
    change_df['DLAC'] = norm['DLAC'] - 1
    change_df['DLAC Imperfect'] = norm['DLAC Imperfect'] - 1
    change_df['SLAC'] = norm['SLAC'] - 1

    percent_change_df = change_df.copy()
    percent_change_df['CEM'] = change_df['CEM'] * 100
    percent_change_df['PF'] = change_df['PF'] * 100
    percent_change_df['DLAC'] = change_df['DLAC'] * 100
    percent_change_df['DLAC Imperfect'] = change_df['DLAC Imperfect'] * 100
    percent_change_df['SLAC'] = change_df['SLAC'] * 100

    # remove decimal point
    percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
                        percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].round(2)
    percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
        percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(str) + '%'

    return percent_change_df


In [ ]:
consumer_payments_dic

{'Thermal_Base': {'CEM': '1.8035e+10',
  'PF': '1.8222e+10',
  'DLAC': '1.7447e+10',
  'DLAC Imperfect': '1.9132e+10',
  'SLAC': '1.5702e+10'},
 '2_Hr_BESS': {'CEM': '2.2151e+10',
  'PF': '2.2329e+10',
  'DLAC': '2.1731e+10',
  'DLAC Imperfect': '2.0867e+10',
  'SLAC': '2.0191e+10'},
 '2_Hr_BESS_Fuelx2': {'CEM': '2.5428e+10',
  'PF': '2.9155e+10',
  'DLAC': '2.3574e+10',
  'DLAC Imperfect': '2.3821e+10',
  'SLAC': '2.3820e+10'},
 '4_Hr_BESS': {'CEM': '2.2078e+10',
  'PF': '2.2916e+10',
  'DLAC': '2.2419e+10',
  'DLAC Imperfect': '2.1373e+10',
  'SLAC': '2.1041e+10'},
 '4_Hr_BESS_Fuelx2': {'CEM': '2.5194e+10',
  'PF': '2.9137e+10',
  'DLAC': '2.6477e+10',
  'DLAC Imperfect': '2.3919e+10',
  'SLAC': '2.4113e+10'},
 '4_Hr_BESS_Fuelx3': {'CEM': '2.6982e+10',
  'PF': '2.7474e+10',
  'DLAC': '2.6470e+10',
  'DLAC Imperfect': '2.6184e+10',
  'SLAC': '2.4748e+10'},
 '4_Hr_BESS_Fuelx4': {'CEM': '2.8275e+10',
  'PF': '2.9121e+10',
  'DLAC': '2.9967e+10',
  'DLAC Imperfect': '3.0827e+10',
  'SLAC

In [ ]:
# Create a dataframe from consumer_payments_dic with rows for each case_name in case_names
consumer_payments_df = pd.DataFrame.from_dict(consumer_payments_dic, orient='index').reset_index()
consumer_payments_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
consumer_payments_df.to_csv(tables_path + 'consumer_payments.csv', index=False)
consumer_payments_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,1.8035e+10,1.8222e+10,1.7447e+10,1.9132e+10,1.5702e+10
1,2_Hr_BESS,2.2151e+10,2.2329e+10,2.1731e+10,2.0867e+10,2.0191e+10
2,2_Hr_BESS_Fuelx2,2.5428e+10,2.9155e+10,2.3574e+10,2.3821e+10,2.3820e+10
3,4_Hr_BESS,2.2078e+10,2.2916e+10,2.2419e+10,2.1373e+10,2.1041e+10
4,4_Hr_BESS_Fuelx2,2.5194e+10,2.9137e+10,2.6477e+10,2.3919e+10,2.4113e+10
5,4_Hr_BESS_Fuelx3,2.6982e+10,2.7474e+10,2.6470e+10,2.6184e+10,2.4748e+10
6,4_Hr_BESS_Fuelx4,2.8275e+10,2.9121e+10,2.9967e+10,3.0827e+10,2.9284e+10
7,6_Hr_BESS,2.2028e+10,2.3918e+10,2.2539e+10,2.1149e+10,2.0931e+10


In [ ]:
consumer_payments_change_df = percent_change_df(consumer_payments_df)
consumer_payments_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,-1.03%,0.0%,-4.25%,4.99%,-13.83%
1,2_Hr_BESS,-0.8%,0.0%,-2.68%,-6.55%,-9.57%
2,2_Hr_BESS_Fuelx2,-12.78%,0.0%,-19.14%,-18.3%,-18.3%
3,4_Hr_BESS,-3.66%,0.0%,-2.17%,-6.73%,-8.18%
4,4_Hr_BESS_Fuelx2,-13.53%,0.0%,-9.13%,-17.91%,-17.24%
5,4_Hr_BESS_Fuelx3,-1.79%,0.0%,-3.65%,-4.7%,-9.92%
6,4_Hr_BESS_Fuelx4,-2.91%,0.0%,2.91%,5.86%,0.56%
7,6_Hr_BESS,-7.9%,0.0%,-5.77%,-11.58%,-12.49%


In [ ]:
# Create a new dataframe by dividing the 'DLAC', 'DLAC Imperfect', and 'SLAC' columns by the 'CEM' column
consumer_payments_ratio_df = consumer_payments_df.copy()
consumer_payments_ratio_df['CEM'] = consumer_payments_df['CEM'].astype(float) / consumer_payments_df['CEM'].astype(float) - 1
consumer_payments_ratio_df['PF'] = consumer_payments_df['PF'].astype(float) / consumer_payments_df['CEM'].astype(float) - 1
consumer_payments_ratio_df['DLAC'] = consumer_payments_df['DLAC'].astype(float) / consumer_payments_df['CEM'].astype(float) -1 
consumer_payments_ratio_df['DLAC Imperfect'] = consumer_payments_df['DLAC Imperfect'].astype(float) / consumer_payments_df['CEM'].astype(float) -1
consumer_payments_ratio_df['SLAC'] = consumer_payments_df['SLAC'].astype(float) / consumer_payments_df['CEM'].astype(float) -1

consumer_payments_ratio_df['CEM'] = consumer_payments_ratio_df['CEM'].round(2) * 100
consumer_payments_ratio_df['PF'] = consumer_payments_ratio_df['PF'].round(2) * 100
consumer_payments_ratio_df['DLAC'] = consumer_payments_ratio_df['DLAC'].round(2) * 100
consumer_payments_ratio_df['DLAC Imperfect'] = consumer_payments_ratio_df['DLAC Imperfect'].round(2) * 100
consumer_payments_ratio_df['SLAC'] = consumer_payments_ratio_df['SLAC'].round(2) * 100

# Remove the decimal point from the normalized total system costs
consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(int)

# Add a '%' symbol at the end of the normalized total system costs
consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(str) + '%'

consumer_payments_ratio_df.to_csv(tables_path + 'consumer_payments_ratio.csv', index=False)
consumer_payments_ratio_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0%,1%,-3%,6%,-13%
1,2_Hr_BESS,0%,1%,-2%,-6%,-9%
2,2_Hr_BESS_Fuelx2,0%,15%,-7%,-6%,-6%
3,4_Hr_BESS,0%,4%,2%,-3%,-5%
4,4_Hr_BESS_Fuelx2,0%,16%,5%,-5%,-4%
5,4_Hr_BESS_Fuelx3,0%,2%,-2%,-3%,-8%
6,4_Hr_BESS_Fuelx4,0%,3%,6%,9%,4%
7,6_Hr_BESS,0%,9%,2%,-4%,-5%


In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

# BEGIN: Add unit profit tables to PDF

# create a date stamp of just the year, month and day
date = datetime.datetime.now()
date = date.strftime("%Y-%m-%d-%H-%M")

In [ ]:
# loop through the unit_profits_dic, and change the names in the Resources column according to the dictionary
for case_name in case_names:
    unit_profit_dic[case_name] = replace_strings_in_df_column(unit_profit_dic[case_name], 'Resource', simplify_resource_names)

In [ ]:
unit_profits_fln =  'unit_profits_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + unit_profits_fln) as pdf:
    for case_name in case_names:
        # Get the unit profit dataframe for the current case
        unit_profit_df = unit_profit_dic[case_name]

        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Hide the axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_frame_on(False)

        # Create a table
        table = ax.table(cellText=unit_profit_df.values, colLabels=unit_profit_df.columns, cellLoc='center', loc='center')

        # Set table style
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)

        # Add a title
        ax.set_title(f'Unit Profits for {case_name} ($/MW)', fontsize=14, fontweight='bold')

        # Add the figure to the PDF
        pdf.savefig(fig)

        # Close the figure
        plt.close(fig)
# END: Add unit profit tables to PDF

In [ ]:
percentage_profits_fln =  'percentage_profits_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + percentage_profits_fln) as pdf:
    for case_name in case_names:
        # Get the unit profit dataframe for the current case
        percentage_profits_df = percentage_profits_dic[case_name]

        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Hide the axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_frame_on(False)

        # Create a table
        table = ax.table(cellText=percentage_profits_df.values, \
                         colLabels=percentage_profits_df.columns, \
                            cellLoc='center', loc='center')

        # Set table style
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)

        # Add a title
        ax.set_title(f'% Profits over Fixed Costs for {case_name}', fontsize=14, fontweight='bold')

        # Add the figure to the PDF
        pdf.savefig(fig)

        # Close the figure
        plt.close(fig)
# END: Add unit profit tables to PDF

In [ ]:
# print 4 hour battery case percentage profits to csv
percentage_profits_dic['4_Hr_BESS'].to_csv(tables_path + '4_hour_battery_case_percentage_profits.csv', index=False)
percentage_profits_dic['4_Hr_BESS_Fuelx4'].to_csv(tables_path + '4_hour_battery_case_fuelx4_percentage_profits.csv', index=False)

In [ ]:
battery_df = pd.DataFrame()
for key in percentage_profits_dic:
    print(f"Key: {key}")
    # print(percentage_profits_dic[key])
    battery_row = percentage_profits_dic[key][percentage_profits_dic[key]['Resource'] == 'Battery']
    battery_row.insert(0, 'Case', key)
    battery_df = pd.concat([battery_df, battery_row], ignore_index=True)
    if battery_df.empty:
        battery_df = None
    # print(battery_df)

battery_df.to_csv(tables_path + 'battery_percentage_profits.csv', index=False)

Key: Thermal_Base
Key: 2_Hr_BESS
Key: 2_Hr_BESS_Fuelx2
Key: 4_Hr_BESS
Key: 4_Hr_BESS_Fuelx2
Key: 4_Hr_BESS_Fuelx3
Key: 4_Hr_BESS_Fuelx4
Key: 6_Hr_BESS


AttributeError: 'NoneType' object has no attribute 'to_csv'

In [ ]:
# # Convert the percentage profits for 'Thermal_Case' to a DataFrame
# thermal_case_df = percentage_profits_dic['Thermal_Case']

# # Define the filename
# thermal_case_csv_filename = pdf_path + 'thermal_case_percentage_profits.csv'

# # Save the DataFrame to a CSV file
# thermal_case_df.to_csv(thermal_case_csv_filename, index=False)

In [ ]:
consumer_payments_fln =  'consumer_payments_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + consumer_payments_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=consumer_payments_df.values, colLabels=consumer_payments_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Consumer Payments for Each Case ($)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

ErrorException: syntax: character literal contains multiple characters

In [ ]:
consumer_payments_ratio_fln =  'consumer_payments_ratio_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + consumer_payments_ratio_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=consumer_payments_ratio_df.values, colLabels=consumer_payments_ratio_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Ratio of Consumer Payments (Normalized by CEM)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

ErrorException: syntax: character literal contains multiple characters

In [ ]:
pmr_cpr_dic = {}

for case_name in case_names:
    percentage_profits_df = percentage_profits_dic[case_name]
    consumer_payments_ratio = consumer_payments_ratio_df[consumer_payments_ratio_df['Case Name'] == case_name].iloc[0]
    
    # Create a new row with the consumer payments ratio
    new_row = pd.DataFrame({
        'Resource': ['Consumer Payments Ratio'],
        'CEM': [f"{float(consumer_payments_ratio['CEM']) * 100:.2f}%"],
        'DLAC': [f"{float(consumer_payments_ratio['DLAC']) * 100:.2f}%"],
        'DLAC Imperfect': [f"{float(consumer_payments_ratio['DLAC Imperfect']) * 100:.2f}%"],
        'SLAC': [f"{float(consumer_payments_ratio['SLAC']) * 100:.2f}%"]
    })
    
    # Append the new row to the dataframe
    percentage_profits_df = pd.concat([percentage_profits_df, new_row], ignore_index=True)
    
    # Update the dictionary
    pmr_cpr_dic[case_name] = percentage_profits_df

# pmr_cpr_dic
# concatenate all the dataframes in pmr_cpr_dic, adding a row for the name of the case
pmr_cpr_combined_df = pd.DataFrame()

for case_name, df in pmr_cpr_dic.items():
    df.insert(0, 'Case Name', case_name)
    pmr_cpr_combined_df = pd.concat([pmr_cpr_combined_df, df], ignore_index=True)


# replace the names of the cases with the print names
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('2_Hour_Batteries_Case', '2 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('2_Hour_Batteries_Case_Fuelx2', '2 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case', '4 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case_Fuelx2', '4 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case_Fuelx3', '4 Hr Bat, Fuel Costs x3')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case_Fuelx4', '4 Hr Bat, Fuel Costs x4')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('6_Hour_Batteries_Case', '6 Hour Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('6_Hour_Batteries_Case_Fuelx2', '6 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('8_Hour_Batteries_Case', '8 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('8_Hour_Batteries_Case_Fuelx2', '8 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('10_Hour_Batteries_Case', '10 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('10_Hour_Batteries_Case_Fuelx2', '10 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('Thermal_Case', 'Thermal')

pmr_cpr_combined_df.to_csv(tables_path + 'pmr_cpr_combined_' + date + '.csv', index=False)

ErrorException: syntax: { } vector syntax is discontinued around c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y111sZmlsZQ==.jl:1

In [ ]:

total_system_costs_fln =  'total_system_costs_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + total_system_costs_fln) as pdf:

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_system_costs_df.values, colLabels=total_system_costs_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title(f'Total Costs for each case ($)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)
# END: Add unit profit tables to PDF

ErrorException: syntax: character literal contains multiple characters

In [ ]:
total_nse_df.columns

UndefVarError: UndefVarError: `total_nse_df` not defined

In [ ]:
total_nse_fln =  'total_nse_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + total_nse_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_nse_df.values, colLabels=total_nse_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Total Non-served Energy for Each Case (MW)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

ErrorException: syntax: character literal contains multiple characters

In [ ]:
total_unmet_rsv_fln =  'total_unmet_rsv_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + total_unmet_rsv_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_unmet_rsv_df.values, colLabels=total_unmet_rsv_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Total Unmet Reserves for Each Case (MW)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

ErrorException: syntax: character literal contains multiple characters

In [ ]:
pf_nse_cost = pd.read_csv(pf_results_path + 'revenue_nse_cost.csv', header=None)

In [ ]:
  if pf_ur_cost.shape[0] < pf_ur_cost.shape[1]:
        pf_ur_cost = pf_ur_cost.T

In [ ]:
pf_ur_cost

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
8681,0.0
8682,0.0
8683,0.0
8684,0.0


In [ ]:
pf_sum_ur_cost

4948861.880002092